# Fix pathing

In [1]:
import sys


sys.path.append("..")


In [2]:
import constants

import os


constants.PROJECT_DIRECTORY_PATH = os.path.dirname(os.path.dirname(constants.PROJECT_DIRECTORY_PATH))


# Imports

In [3]:
import datahandler
import utils
import pathing

import osmnx as ox
import networkx
import random
import time
from tqdm import tqdm


# Constants

In [4]:
data_preprocessor = datahandler.DataPreprocessorOUS_V2()
data_preprocessor.execute()

data_loader = datahandler.DataLoader(datahandler.DataPreprocessorOUS_V2)
data_loader.execute(clean=False, processed=False, enhanced=False)

Enhancing dataset: 100%|██████████| 2/2 [00:00<?, ?it/s]


# Methods

In [5]:
def get_node(od: pathing.OriginDestination, x, y):
    if (x, y) in od.node_cache:
        return od.node_cache[(x, y)]
    
    while True:
        node = ox.distance.nearest_nodes(od.graph, x, y)
        if networkx.has_path(od.graph, node, od.node_validator) and networkx.has_path(od.graph, od.node_validator, node):
            od.node_cache[(x, y)] = node
            return node
        else:
            od.graph.remove_node(node)


In [6]:
def gen_linkers(road_type_factors, link_factor):
    linkables = [
        "motorway",
        "trunk",
        "primary",
        "secondary",
        "tertiary",
    ]

    for linkable in linkables:
        if linkable in road_type_factors:
            road_type_factors[linkable + "_link"] = road_type_factors[linkable] * link_factor
    
    return road_type_factors


In [7]:
def evaluate_travel_time_accuracy(benchmark_times, calculated_times):
    total_diff = 0
    total_diff_percentage = 0

    for path, acceptable_range in benchmark_times.items():
        if path in calculated_times:

            calculated_time = calculated_times[path]
            lower_bound, upper_bound = acceptable_range

            # Check if the calculated time falls within the acceptable range
            if lower_bound <= calculated_time <= upper_bound:
                diff = 0
                diff_percentage = 0
            else:
                # If outside the range, find the smallest absolute difference to the bounds
                diff = min(abs(calculated_time - lower_bound), abs(calculated_time - upper_bound))
                diff_percentage = min((diff / lower_bound) * 100, (diff / upper_bound) * 100)

            total_diff += diff
            total_diff_percentage += diff_percentage

            print(f"Path {path}:")
            print(f"  Benchmark Time: {acceptable_range} minutes")
            print(f"  Calculated Time: {calculated_time} minutes")
            print(f"  Difference: {diff} minutes ({diff_percentage:.2f}%)")
            print()
        else:
            print(f"Path {path} not found in calculated times.")

    avg_diff_percentage = total_diff_percentage / len(benchmark_times)
    print(f"Total Discrepancy: {total_diff} minutes across all paths.")
    print(f"Average Discrepancy Percentage: {avg_diff_percentage:.2f}%")


In [8]:
def calculate_error(od: pathing.OriginDestination, benchmark_times):
    calculated_times = {}
    
    # Assuming utils and get_node functions are defined and work as intended
    for ((start_lon, start_lat), (end_lon, end_lat)), _ in benchmark_times.items():
        # Convert geographic coordinates to UTM and get the corresponding node
        start_x, start_y = utils.geographic_to_utm(start_lon, start_lat)
        end_x, end_y = utils.geographic_to_utm(end_lon, end_lat)

        start_node = get_node(od, start_x, start_y)
        end_node = get_node(od, end_x, end_y)

        # Calculate the shortest path and total travel time
        shortest_time_path = ox.shortest_path(
            od.graph,
            start_node,
            end_node,
            weight='time'
        )
        total_travel_time = sum(od.graph[u][v][0]['time'] for u, v in zip(shortest_time_path[:-1], shortest_time_path[1:]))

        # Store the calculated travel time
        calculated_times[((start_lon, start_lat), (end_lon, end_lat))] = total_travel_time

    # Calculate MSE
    errors = []

    for path, acceptable_range in benchmark_times.items():
        calculated_time = calculated_times[path]
        lower_bound, upper_bound = acceptable_range

        # Check if the calculated time falls within the acceptable range
        if lower_bound <= calculated_time <= upper_bound:
            error = 0
        else:
            # If outside the range, find the smallest absolute difference to the bounds
            error = min(abs(calculated_time - lower_bound), abs(calculated_time - upper_bound))
        
        errors.append(error)

    mse = sum(error ** 2 for error in errors) / len(errors)
    
    return mse, calculated_times


In [9]:
def optimize_parameters(od: pathing.OriginDestination, benchmark_times, max_duration_minutes):
    start_time = time.time()
    max_duration_seconds = max_duration_minutes * 60
    
    # Define parameter bounds
    param_bounds = {
        "default_intersection_penalty": (1, 15),
        "traffic_signal_penalty": (1, 15),
        "linkable_factor": (0.8, 1.0),
        "road_type_factors": {
            "motorway": (0.9, 1.0),
            "trunk": (0.9, 1.0),
            "primary": (0.7, 0.9),
            "secondary": (0.7, 0.9),
            "tertiary": (0.7, 0.9),
            "unclassified": (0.6, 0.8),
            "residential": (0.5, 0.7),
            "living_street": (0.3, 0.5)
        }
    }
    
    # Initialize the best solution
    best_solution = None
    best_error = float("inf")
    
    # Progress bar
    with tqdm(desc="Optimizing Parameters") as pbar:
        while time.time() - start_time < max_duration_seconds:
            # Generate a random solution within the bounds
            current_solution = {
                "default_intersection_penalty": random.randint(*param_bounds["default_intersection_penalty"]),
                "traffic_signal_penalty": random.randint(*param_bounds["traffic_signal_penalty"]),
                "road_type_factors": {k: random.uniform(*v) for k, v in param_bounds["road_type_factors"].items()}
            }
            linkable_factor = random.uniform(*param_bounds["linkable_factor"])

            current_solution["road_type_factors"] = gen_linkers(current_solution["road_type_factors"], linkable_factor)
            
            # Set graph weights with the current solution
            od.set_graph_weights_v2(**current_solution)
            
            # Calculate the error for the current solution
            current_error, _ = calculate_error(od, benchmark_times)
            
            # Update best solution if the current one is better
            if current_error < best_error:
                best_solution = current_solution
                best_linkable_factor = linkable_factor
                best_error = current_error
                tqdm.write(
                    f"    New best solution:\n"
                    f"        Default Intersection Penalty: {best_solution['default_intersection_penalty']}\n"
                    f"        Traffic Signal Penalty: {best_solution['traffic_signal_penalty']}\n"
                    f"        Linkable Factor: {best_linkable_factor}\n"
                    f"        Road Type Factors:\n"
                    + "\n".join([f"            {k}: {v}" for k, v in best_solution['road_type_factors'].items()]) +
                    f"\n        Error: {best_error}"
                )
            
            # Update progress bar
            pbar.update()

            # early stop
            if current_error == 0:
                break
    
    return best_solution, best_error


In [10]:
def fix_lat_lon(benchmark_times):
    new_benchmark_times = {}

    for ((start_lat, start_lon), (end_lat, end_lon)), benchmark in benchmark_times.items():
        new_benchmark_times[((start_lon, start_lat), (end_lon, end_lat))] = benchmark

    return new_benchmark_times

# Main

In [11]:
od = pathing.OriginDestination(
    dataset_id="oslo",
    utm_epsg=f"EPSG:326{33}"
)

od.get_graph()

central_depot_grid_id = 22620006649000
central_depot_x, central_depot_y = utils.id_to_utm(central_depot_grid_id)
od.node_validator = ox.distance.nearest_nodes(od.graph, central_depot_x, central_depot_y)


In [15]:
# times from Google Maps (Sunday, start at 02:00)
benchmark_times = {
    ((59.9369806, 10.7343661), (59.6800045, 10.6572104)): (35, 45),
    ((59.93596, 10.73244), (59.92629, 10.77570)): (4, 9),
    ((59.90864, 10.73921), (59.93037, 10.77273)): (9, 16),
    ((59.92727, 10.73174), (59.86305, 10.66617)): (50, 60),
    ((59.82052, 10.47168), (59.95577, 11.04773)): (40, 50),
    ((60.00352, 10.76216), (59.83905, 10.80742)): (28, 28),

    ((60.31262, 11.14674), (60.11964, 11.47577)): (40, 40),
    ((60.11964, 11.47577), (60.13423, 11.166858)): (22, 26),
    ((60.13423, 11.166858), (59.879044, 11.561687)): (50, 50),
    ((59.879044, 11.561687), (59.932076, 10.987775)): (40, 40),
    ((59.932076, 10.987775), (60.042152, 10.880784)): (20, 22),
    ((60.042152, 10.880784), (59.930733, 10.831094)): (20, 20),
    ((59.930733, 10.831094), (59.917, 10.758744)): (9, 14),
    ((59.917, 10.758744), (59.93917, 10.741926)): (7, 14),
    ((59.93917, 10.741926), (59.71566, 10.853331)): (35, 35),
    ((59.71566, 10.853331), (59.659687, 10.725881)): (18, 18),
    ((59.659687, 10.725881), (59.833073, 10.806946)): (22, 22),
    ((59.833073, 10.806946), (59.830055, 10.439887)): (30, 40),
    ((59.830055, 10.439887), (59.89813, 10.509723)): (14, 14),
    ((59.89813, 10.509723), (59.939663, 10.687367)): (16, 20),
    ((59.939663, 10.687367), (59.893173, 10.806364)): (12, 16),
    ((59.893173, 10.806364), (59.960304, 10.884091)): (12, 18),
    ((59.960304, 10.884091), (59.997875, 11.03928)): (20, 20),
    ((59.997875, 11.03928), (59.917873, 10.585751)): (28, 35),
    ((59.917873, 10.585751), (60.31262, 11.14674)): (55, 65),
}

benchmark_times = fix_lat_lon(benchmark_times)


In [16]:
best_solution, best_error = optimize_parameters(
    od,
    benchmark_times,
    max_duration_minutes=420
)


Optimizing Parameters: 0it [00:00, ?it/s]

Optimizing Parameters: 1it [00:10, 10.36s/it]

    New best solution:
        Default Intersection Penalty: 14
        Traffic Signal Penalty: 2
        Linkable Factor: 0.8827425342942617
        Road Type Factors:
            motorway: 0.9301923340748466
            trunk: 0.9278754271707412
            primary: 0.7134005967671256
            secondary: 0.7061657944818038
            tertiary: 0.835539790384278
            unclassified: 0.6167638947426751
            residential: 0.698745246855365
            living_street: 0.3498060314750464
            motorway_link: 0.8211203383623247
            trunk_link: 0.8190751060900708
            primary_link: 0.6297490507572512
            secondary_link: 0.6233625830527884
            tertiary_link: 0.7375665120675138
        Error: 12.508052191914619


Optimizing Parameters: 2it [00:19,  9.77s/it]

    New best solution:
        Default Intersection Penalty: 11
        Traffic Signal Penalty: 5
        Linkable Factor: 0.8818405704433173
        Road Type Factors:
            motorway: 0.9722804896990507
            trunk: 0.9820489683880131
            primary: 0.7114713449372677
            secondary: 0.8103050474834286
            tertiary: 0.855243256658168
            unclassified: 0.6174152737498894
            residential: 0.6375312071114709
            living_street: 0.4323558034783087
            motorway_link: 0.8573963816671187
            trunk_link: 0.8660106224865568
            primary_link: 0.6274042966735544
            secondary_link: 0.714559865305886
            tertiary_link: 0.7541882013192394
        Error: 7.661986585632746


...

Optimizing Parameters: 611it [1:40:26,  9.79s/it]

    New best solution:
        Default Intersection Penalty: 3
        Traffic Signal Penalty: 9
        Linkable Factor: 0.8852785159295735
        Road Type Factors:
            motorway: 0.9002939172528034
            trunk: 0.908734898168141
            primary: 0.8339970492490043
            secondary: 0.896083563077741
            tertiary: 0.8269732787830024
            unclassified: 0.7768291076328838
            residential: 0.558714278383741
            living_street: 0.3167512865766974
            motorway_link: 0.797010862965984
            trunk_link: 0.804483482023704
            primary_link: 0.738319670048802
            secondary_link: 0.7932835268703469
            tertiary_link: 0.7321016769544298
        Error: 2.269080052208785


Optimizing Parameters: 758it [2:05:23, 10.06s/it]

    New best solution:
        Default Intersection Penalty: 4
        Traffic Signal Penalty: 9
        Linkable Factor: 0.8521506076850045
        Road Type Factors:
            motorway: 0.9004725950459835
            trunk: 0.9602555300158857
            primary: 0.8957176686384549
            secondary: 0.812242943390013
            tertiary: 0.8779790728890131
            unclassified: 0.7187557285185038
            residential: 0.5840639670583821
            living_street: 0.3375975835998158
            motorway_link: 0.7673382690721279
            trunk_link: 0.818282333435923
            primary_link: 0.7632863556444549
            secondary_link: 0.6921533177976563
            tertiary_link: 0.7481704004970894
        Error: 2.2271426282859084


Optimizing Parameters: 1313it [8:01:07, 21.99s/it]  


In [17]:
od.set_graph_weights_v2(**best_solution)

_, calculated_times = calculate_error(od, benchmark_times)

evaluate_travel_time_accuracy(benchmark_times, calculated_times)


Path ((10.7343661, 59.9369806), (10.6572104, 59.6800045)):
  Benchmark Time: (35, 45) minutes
  Calculated Time: 39.55585198503308 minutes
  Difference: 0 minutes (0.00%)

Path ((10.73244, 59.93596), (10.7757, 59.92629)):
  Benchmark Time: (4, 9) minutes
  Calculated Time: 6.859429199999245 minutes
  Difference: 0 minutes (0.00%)

Path ((10.73921, 59.90864), (10.77273, 59.93037)):
  Benchmark Time: (9, 16) minutes
  Calculated Time: 10.89508881102026 minutes
  Difference: 0 minutes (0.00%)

Path ((10.73174, 59.92727), (10.66617, 59.86305)):
  Benchmark Time: (50, 60) minutes
  Calculated Time: 53.296575198499426 minutes
  Difference: 0 minutes (0.00%)

Path ((10.47168, 59.82052), (11.04773, 59.95577)):
  Benchmark Time: (40, 50) minutes
  Calculated Time: 43.2345763703291 minutes
  Difference: 0 minutes (0.00%)

Path ((10.76216, 60.00352), (10.80742, 59.83905)):
  Benchmark Time: (28, 28) minutes
  Calculated Time: 30.46095427225396 minutes
  Difference: 2.460954272253961 minutes (8.79